In [ ]:
import os, pdb

import argparse
import numpy as np
import torch
import requests
from PIL import Image
import sys
sys.path.append('pix2pix-zero/src')
sys.path.append('pix2pix-zero/src/utils')
from utils.ddim_inv import DDIMInversion
from utils.scheduler import DDIMInverseScheduler
from types import SimpleNamespace 
import matplotlib.pyplot as plt
from glob import glob
import argparse

cls = 'dog'
args = {
    'input_image': f'/grogu/user/akirchme/art3d_sd_webui/fg/{cls}/',
    'results_folder': f'/grogu/user/akirchme/art3d_sd_webui/inversion/{cls}',
    'num_ddim_steps': 100,
    'model_path': 'stabilityai/stable-diffusion-2-1-base',
    'use_float_16': False,
    'prompt': f'{cls}'
}

args = SimpleNamespace(**args)

torch_dtype = torch.float16 if args.use_float_16 else torch.float32   
#pipe = DDIMInversion.from_pretrained(args.model_path, torch_dtype=torch_dtype).to("cuda")
#pipe.scheduler = DDIMInverseScheduler.from_config(pipe.scheduler.config)
    
def generate_inversion(args):
    # make the output folders
    os.makedirs(args.results_folder, exist_ok=True)

    # if the input is a folder, collect all the images as a list
    if os.path.isdir(args.input_image):
        l_img_paths = sorted(glob(os.path.join(args.input_image, "*.png")))
    else:
        l_img_paths = [args.input_image]

    for img_path in l_img_paths:
        bname = os.path.basename(img_path).split(".")[0]
        res_path = os.path.join(args.results_folder, f"{bname}.pt")
        if os.path.exists(res_path):
            continue
        img = Image.open(img_path).resize((512,512), Image.Resampling.LANCZOS)
        res = pipe(
            args.prompt, 
            guidance_scale=1,
            num_inversion_steps=args.num_ddim_steps,
            img=img,
            torch_dtype=torch_dtype,
            return_intermediate=True
        )
        # save the inversion
        torch.save(res, res_path)

In [ ]:
#generate_inversion(args)

In [ ]:
from diffusers import StableDiffusionPipeline, DDIMScheduler, DPMSolverMultistepScheduler

pipe = StableDiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-2-1-base').to('cuda')


In [ ]:
#{os.path.basename(x) : torch.load(x).shape for x in list(sorted(glob(os.path.join(args.results_folder, "*.pt"))))}

In [ ]:
def decode_latents(vae, latents):
    latents = 1 / 0.18215 * latents
    image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    # we always cast to float32 as this does not cause significant overhead and is compatible with bfloa16
    image = image.detach().cpu().permute(0, 2, 3, 1).float().numpy()
    return image

fig, axs = plt.subplots(2, 2)
axs[0,0].imshow(decode_latents(pipe.vae, x_inv[0:1].cuda()).squeeze())
axs[0,1].imshow(decode_latents(pipe.vae, x_inv[-1:].cuda()).squeeze())
axs[1,0].imshow(x_inv[0,:3,:,:].permute(1,2,0))
axs[1,1].imshow(x_inv[-1,:3,:,:].permute(1,2,0))

In [ ]:
x_inv[-1].shape

In [ ]:
prompt = 'photo of a dog, far, side view'
neg_prompt = 'drawing, shiny'
for path in list(sorted(glob(os.path.join(args.results_folder, "*.pt")))):
    bname = os.path.basename(path).split(".")[0]
    x_inv = torch.load(os.path.join(args.results_folder, f"{bname}.pt"))
    fig, axs = plt.subplots(1, 3, figsize=(20,20))
    axs[0].imshow(Image.open(f'/grogu/user/akirchme/art3d_sd_webui/fg/{cls}/{bname}.png'))
    pipe.scheduler = DDIMScheduler.from_pretrained('stabilityai/stable-diffusion-2-1-base', subfolder='scheduler')
    axs[1].imshow(pipe(latents=x_inv[-1].unsqueeze(0), prompt=prompt, negative_prompt=neg_prompt).images[0])
    #axs[2].imshow(pipe(latents=x_inv[].unsqueeze(0), prompt='dog', num_inference_steps=100).images[0])
    pipe.scheduler = DPMSolverMultistepScheduler.from_pretrained('stabilityai/stable-diffusion-2-1-base', subfolder='scheduler')
    axs[2].imshow(pipe(latents=x_inv[-1].unsqueeze(0), prompt=prompt, negative_prompt=neg_prompt).images[0])
    plt.show()